<font size=4>
Author of Question: Kimia Noorbakhsh
			<br/>

In this assignment, you are going to implement a Naive Bayes Classifier for the MNIST Dataset (Well, of course, **from scratch**!). The MNIST data set is a vast database of handwritten digits that is commonly used to form various image processing systems. 

Please note the following before you continue:
- After implementing your Classifier, train your model on the **train** section of the MNIST dataset and validate your model by testing it on the test set.
- Note that if you use any of the **test** images during training or for improving the accuracy, you will not earn any points for this assignment. 
- Do not forget to use **Laplace Smoothing** to avoid overfitting.

Recall Bayes rule:
    $$P(c|x) =  \frac{P(x|c)P(c)}{P(x)} \;\;\;\;(1)$$
    
Here $x$ stands for the image, or more precisely, the pixel values of the formatted image as a vector, and $c$ stands for the number, which can be 0, 1, ..., 9. We can read the left side $P(c|x)$ as "the probability of the class being $c$ given the $x$" data (posterior). We can read the right side $P(x|c)$ as "the probability of $x$ data being in the $c$" class (likelihood). We care about the value of $c$. It tells us "what number" this picture is. The chosen class is simply the one with the highest probability for this data:
$$c^* = argmax_{c}P(c|x)$$
Now, we can ignore $P(x)$ in equation (1) (why?). Using this information, we can simplify our problem so that, in order to choose "which digit" given an image, all we need to do is calculate this argmax (P(x) is removed):
$$c^* = argmax_{c}P(x|c)P(c)$$
Now, we need to think about how to calculate $P(c)$, and $P(x|c)$. We leave this section for you to think about ^_^. But as a guide for $P(x|c)$, read the following. 

Remember that pixels represent the intensity of light, and that the intensity of light is in fact continuous. A first reasonable estimation to model continuous data is the multivariate Gaussian or multivariate Normal. We can write:
$$P(x|c) = \frac{1}{\sqrt{(2\pi)^{D}|\Sigma|}}\exp(-\frac{1}{2}(x - \mu)^{T}\Sigma^{-1}(x-\mu))$$
Note that because probabilities are very small when dimensionality is high, we are going to work with log-probability rather than probability. So instead of getting numbers that are very close to 0, which is inaccurate when you use a computer to represent them, we're just going to get negative numbers. The log-probability can be represented as ($D$ is the dimentionality):
$$\log{P(x|c) = -\frac{D}{2}\ln(2\pi)-\frac{1}{2}\ln|\Sigma|-\frac{1}{2}(x - \mu)^{T}\Sigma^{-1}(x-\mu)}$$
To calculate $\mu$ and $\Sigma$, you can use the **sample** mean and covariance (see [here.](https://en.wikipedia.org/wiki/Sample_mean_and_covariance)) Also note that to get the argmax over $P(x|c)P(c)$, we can choose the digit class using:
$$c^* = argmax_{c}(\log P(x|c)+\log P(c))$$
Now, let's dive into implementing a **Gaussian Naive Bayes Classifier.**

## Loading data

For your convineince, use the following cells to access the data. 

In [11]:
!pip install torchvision
!pip install numpy
# and other libraries you might need

from torchvision import datasets
import numpy as np
from scipy.stats import multivariate_normal

In [12]:
train_data = datasets.MNIST('./data', train=True, download=True)
test_data  = datasets.MNIST('./data', train=False, download=True)

train_images = np.array(train_data.data)
train_labels = np.array(train_data.targets)
test_images = np.array(test_data.data)
test_labels = np.array(test_data.targets)

## Training the Model

In [13]:
class Bayes:
    def train(self, train_images, train_labels):
        # Put your Code here. (30 Points)
        self.labels = set(train_labels)
        flatten_train_images = train_images.reshape(len(train_images), -1)
        self.means = {c: self.sample_mean(flatten_train_images, train_labels, c) for c in self.labels}
        self.covs = {}
        self.label_pros = {}
        self.smoothing = 1
        best_accuracy = 0
        smoothings_test = [1, 10, 100, 500, 1000, 2000, 3000, 4000, 5000, 10000]
        for smoothing in smoothings_test:
            means = {c: self.sample_mean(flatten_train_images, train_labels, c, smoothing) for c in self.labels}
            label_pros = self.calculate_log_p_c(train_labels, smoothing)
            sampled_covs = {c: self.sample_cov(flatten_train_images, train_labels, c, smoothing) for c in
                            self.labels}
            accuracy = self.calc_accuracy(flatten_train_images, train_labels, label_pros, means, sampled_covs)
            print(f'Accuracy on test data with smoothing {smoothing} : {accuracy * 100} %')
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                self.smoothing = smoothing
                self.covs = sampled_covs
                self.label_pros = label_pros
                self.means = means
        print(f'Selected smoothing {self.smoothing}')

    def calc_accuracy(self, images, labels, label_pros=None, means=None, covs=None):
        # Put your Code here. (5 Points)
        flatten_train_images = images.reshape(len(images), -1)
        label_pros = label_pros or self.label_pros
        means = means or self.means
        covs = covs or self.covs
        number_of_corrects = 0
        predicts = self.predict_labels(flatten_train_images, label_pros, means, covs)
        for predicted, label in zip(predicts, labels):
            if predicted == label:
                number_of_corrects += 1
        return number_of_corrects / len(images)

    def predict_labels(self, images, label_pros, means, covs):
        # Put your Code here. (5 Points)
        logpdf = {c: multivariate_normal.logpdf(images, mean=means[c], cov=covs[c]) for c in self.labels}
        predicts = []
        for i in range(len(images)):
            best_label = None
            best_prob = None
            for c in self.labels:
                log_p_x_given_c = logpdf[c][i]
                if (best_prob is None) or ((log_p_x_given_c + label_pros[c]) > best_prob):
                    best_prob = log_p_x_given_c + label_pros[c]
                    best_label = c
            predicts.append(best_label)
        return predicts

    def sample_mean(self, data, train_labels, c, smoothing_parameter=0):
        _sum = np.zeros_like(data[0])
        _length = smoothing_parameter
        for img, label in zip(data, train_labels):
            if label == c:
                _sum += img
                _length += 1
        return _sum / _length

    def sample_cov_with_assume_independence(self, data, train_labels, c, smoothing_parameter=0):
        features_size = len(data[0])
        matrix = np.zeros((features_size, features_size))
        data_with_this_class = np.array([img for img, label in zip(data, train_labels) if label == c])
        for i in range(features_size):
            matrix[i][i] = np.cov(data_with_this_class[:, i]) + smoothing_parameter
        return matrix

    def sample_cov(self, data, train_labels, c, smoothing_parameter=0):
        # because without independence assumption result and accuracy is better we use it
        features_size = len(data[0])
        data_with_this_class = np.array([img for img, label in zip(data, train_labels) if label == c])
        matrix = np.cov(data_with_this_class.transpose())
        return matrix + (np.eye(features_size) * smoothing_parameter)

    def calculate_log_p_c(self, train_labels, smoothing_parameter=0):
        p_c_map = {_: 0 for _ in range(10)}
        for item in train_labels:
            p_c_map[item] += 1
        for k, v in p_c_map.items():
            p_c_map[k] = np.log((v + smoothing_parameter) / (
                    len(train_labels) + smoothing_parameter * len(p_c_map)))  # laplace smoothing
        return p_c_map


In [14]:
network = Bayes()
network.train(train_images, train_labels)

Accuracy on test data with smoothing 1 : 86.61999999999999 %
Accuracy on test data with smoothing 10 : 90.125 %
Accuracy on test data with smoothing 100 : 93.50166666666667 %
Accuracy on test data with smoothing 500 : 95.15 %
Accuracy on test data with smoothing 1000 : 95.56666666666666 %
Accuracy on test data with smoothing 2000 : 95.78333333333333 %
Accuracy on test data with smoothing 3000 : 95.72666666666667 %
Accuracy on test data with smoothing 4000 : 95.59 %
Accuracy on test data with smoothing 5000 : 95.435 %
Accuracy on test data with smoothing 10000 : 94.54833333333333 %
Selected smoothing 2000


## Model Evaluation

In [15]:
print("Accuracy on test data (%) : " + str(network.calc_accuracy(test_images, test_labels) * 100))

Accuracy on test data (%) : 95.19
